In [1]:
import os
os.environ["SPARK_HOME"] = "/Users/zouhairhajji/Documents/dev/spark-2.4.0-bin-hadoop2.7" 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, DoubleType
import operator

import re

spark = SparkSession.builder \
        .master('local[*]') \
        .appName('Matrix multiplication') \
        .config("spark.driver.memory", "2g") \
        .enableHiveSupport() \
        .getOrCreate()


In [3]:
file_regex = '../inputs/*.demo'

# Spark algorithm

In [4]:
flat_data = spark.sparkContext.wholeTextFiles(file_regex)

In [5]:
files = flat_data.map(lambda x: x[0].split('/')[-1]).collect()

In [10]:
def filter_appropriate_words(x):
    splitet_str = re.split('\n| |\'', x)
    
    splitet_str = list(filter(None, splitet_str))
    splitet_str = list(filter(lambda x: False if len(x) < 3 else True, splitet_str))
    
    return splitet_str

formated_input = flat_data    \
    .map(lambda x: (x[0].split('/')[-1],  filter_appropriate_words(x[1])    ))    \
    .flatMap(lambda x: [ (x[0], word) for word in x[1]]  )


result = formated_input     \
    .cartesian(formated_input)   \
    .filter(lambda x: x[0][0] == x[1][0])  \
    .filter(lambda x: x[0][1] != x[1][1])  \
    .map(lambda x:  ( (x[0][0], x[0][1], x[1][1]), 1)    ) \
    .reduceByKey(lambda x, y: x+y)   \
    .map(lambda x: (x[0][0], (x[0][1], x[0][2]), x[1]-1))  

In [14]:
result.saveAsTextFile('output_spark')

In [ ]:
11 21

In [2]:
sc = spark.sparkContext

In [49]:
pairs = sc.parallelize([("a", 1), ("a", 5), ("b", 2), ("a", 8)]) 
pairs.reduceByKey(lambda x, y: y+1).collect()



def T(x):
    if x < 4:
        return [x]
    else :
        return []


dpairs = pairs.flatMap(lambda x: T(x[1])).map(lambda x: (x ,1))

print( pairs.collect() )
print( dpairs.collect() )

[('a', 1), ('a', 5), ('b', 2), ('a', 8)]
[(1, 1), (2, 1)]


In [50]:
pairs .cogroup(dpairs).map(lambda x: (x[0], (list(x[1][0]), list(x[1][1]))   )).collect()

[('a', ([1, 5, 8], [])), (1, ([], [1])), (2, ([], [1])), ('b', ([2], []))]